In [3]:
import geopandas as gpd

# Cluster Sample

In [1]:
from cluster_sampling import ClusterSampler

In [ ]:
ADMIN_IDS = {
    'STATEFP': 'state',
    'STATE_NAME': 'state',
    'COUNTYFP': 'county',
    'COUNTY_NAME': 'county'
}

for label in ['treecover']:

    data_path = f"/home/libe2152/optimizedsampling/0_data/admin_gdfs/usavars/{label}/gdf_counties_2015.geojson"
    gdf = gpd.read_file(data_path)

    out_path = f'/home/libe2152/optimizedsampling/0_data/initial_samples/usavars/{label}/cluster_sampling'

    country_shape_file = '/home/libe2152/optimizedsampling/0_data/boundaries/us/us_states_provinces/ne_110m_admin_1_states_provinces.shp'
    exclude_names = ['Alaska', 'Hawaii', 'Puerto Rico']

    strata_col = 'STATEFP'
    cluster_col = 'COUNTYFP'

    sampler = ClusterSampler(gdf, id_col='id', strata_col=strata_col, cluster_col=cluster_col, ADMIN_IDS=ADMIN_IDS)

    for points_per_cluster in [2]:
        sampler.cluster_col = cluster_col
        sampler.merge_small_strata(points_per_cluster)
        sampler.merge_small_clusters(points_per_cluster)
        for total_sample_size in range(850, 1000, 10):
            
            for seed in [1, 42, 123, 456, 789, 1234, 5678, 9101, 1213, 1415]:
                try:
                    sampler.sample(total_sample_size, points_per_cluster, seed=seed)
                    sampler.save_sampled_ids(out_path)
                    sampler.plot(country_shape_file=country_shape_file, exclude_names=exclude_names)
                except Exception as e:
                    print(e)
                sampler.reset_sample()

NameError: name 'gpd' is not defined

# Convenience Sample

In [4]:
from convenience_sampling import ConvenienceSampler

In [5]:
id_col = 'id'
for label in ['population', 'treecover']:
    gdf_path = f"/home/libe2152/optimizedsampling/0_data/admin_gdfs/usavars/{label}/gdf_counties_2015.geojson"
    gdf = gpd.read_file(gdf_path)

    n_urban = 50
    pop_col = 'POP'
    gdf_urban_path = '/home/libe2152/optimizedsampling/0_data/boundaries/us/us_urban_area_census_2020/tl_2020_us_uac20_with_pop.shp'
    gdf_urban = gpd.read_file(gdf_urban_path)

    distances_dir = f'/home/libe2152/optimizedsampling/0_data/distances/usavars/{label}/distance_to_top50_urban.pkl'

    country_shape_file = '/home/libe2152/optimizedsampling/0_data/boundaries/us/us_states_provinces/ne_110m_admin_1_states_provinces.shp'
    exclude_names = ['Alaska', 'Hawaii', 'Puerto Rico']

    out_path = f'/home/libe2152/optimizedsampling/0_data/initial_samples/usavars/{label}/convenience_sampling'

    print("Reading GeoDataFrame...")
    gdf = gpd.read_file(gdf_path)

    method = 'deterministic'
    for desired_sample_size in range(10, 1000, 10):
        print("Initializing ConvenienceSampler...")
        sampler = ConvenienceSampler(
            id_col=id_col,
            gdf_points=gdf,
            gdf_urban=gdf_urban,
            n_urban=n_urban,
            pop_col=pop_col,
            distances_dir=distances_dir
        )

        sampler.sample(n_samples=desired_sample_size, method=method, seed=1) #seed needed to break ties
        sampler.save_sampled_ids(out_path)
        sampler.plot(country_shape_file=country_shape_file, exclude_names=exclude_names)

    for method in ['probabilistic']:
        for desired_sample_size in range(10, 100, 10):
            for seed in [1, 42, 123, 456, 789, 1234, 5678, 9101, 1213, 1415]:

                print("Initializing ConvenienceSampler...")
                sampler = ConvenienceSampler(
                    id_col=id_col,
                    gdf_points=gdf,
                    gdf_urban=gdf_urban,
                    n_urban=n_urban,
                    pop_col=pop_col,
                    distances_dir=distances_dir
                )

                sampler.sample(n_samples=desired_sample_size, method=method, seed=seed)
                sampler.save_sampled_ids(out_path)
                sampler.plot(country_shape_file=country_shape_file, exclude_names=exclude_names)

Reading GeoDataFrame...
Initializing ConvenienceSampler...
Loading precomputed distances from /home/libe2152/optimizedsampling/0_data/distances/usavars/population/distance_to_top50_urban.pkl...
Running deterministic sampling...
Saved 10 sampled IDs to /home/libe2152/optimizedsampling/0_data/initial_samples/usavars/population/convenience_sampling/deterministic/IDS_top50_urban_10_points_deterministic.pkl
Saved plot to /home/libe2152/optimizedsampling/0_data/initial_samples/usavars/population/convenience_sampling/plots/top50_urban_areas_10_points_deterministic.png
Initializing ConvenienceSampler...
Loading precomputed distances from /home/libe2152/optimizedsampling/0_data/distances/usavars/population/distance_to_top50_urban.pkl...
Running deterministic sampling...
Saved 20 sampled IDs to /home/libe2152/optimizedsampling/0_data/initial_samples/usavars/population/convenience_sampling/deterministic/IDS_top50_urban_20_points_deterministic.pkl
Saved plot to /home/libe2152/optimizedsampling/0_d

# Random Sampling

In [6]:
from random_sampling import RandomSampler

In [7]:
for label in ['population', 'treecover']:

    data_path = f"/home/libe2152/optimizedsampling/0_data/admin_gdfs/usavars/{label}/gdf_counties_2015.geojson"
    gdf = gpd.read_file(data_path)

    out_path = f'/home/libe2152/optimizedsampling/0_data/initial_samples/usavars/{label}/random_sampling'

    country_shape_file = '/home/libe2152/optimizedsampling/0_data/boundaries/us/us_states_provinces/ne_110m_admin_1_states_provinces.shp'
    exclude_names = ['Alaska', 'Hawaii', 'Puerto Rico']

    sampler = RandomSampler(gdf, id_col="id")

    for total_sample_size in range(10, 1000, 10):
        for seed in [1, 42, 123, 456, 789, 1234, 5678, 9101, 1213, 1415]:
            sampler.sample(total_sample_size=total_sample_size, seed=seed)
            sampler.save_sampled_ids(out_path)
            sampler.plot(country_shape_file, exclude_names=["Alaska", "Hawaii", "Puerto Rico"])
            sampler.reset_sample()

[Random Sample] Sampled 10 random points.
[Random Sample] Saved IDs to /home/libe2152/optimizedsampling/0_data/initial_samples/usavars/population/random_sampling/random_sample_10_points_seed_1.pkl
[Random Sample] Sampled 10 random points.
[Random Sample] Saved IDs to /home/libe2152/optimizedsampling/0_data/initial_samples/usavars/population/random_sampling/random_sample_10_points_seed_42.pkl
[Random Sample] Sampled 10 random points.
[Random Sample] Saved IDs to /home/libe2152/optimizedsampling/0_data/initial_samples/usavars/population/random_sampling/random_sample_10_points_seed_123.pkl
[Random Sample] Sampled 10 random points.
[Random Sample] Saved IDs to /home/libe2152/optimizedsampling/0_data/initial_samples/usavars/population/random_sampling/random_sample_10_points_seed_456.pkl
[Random Sample] Sampled 10 random points.
[Random Sample] Saved IDs to /home/libe2152/optimizedsampling/0_data/initial_samples/usavars/population/random_sampling/random_sample_10_points_seed_789.pkl
[Random 